# Day 2, ch.3: Decorator
## 1) function 레벨에서 데코레이터 사용하기: parameter가 없는 경우
### decorator란?
- decorator란?: 데코레이터를 적용한 함수를 wrapping 하고, 이 wrapping 된 함수의 앞뒤에 추가적으로 꾸며질 구문 들을 정의해서 손쉽게 재사용 가능하게 해주는 것
- 아래 샘플 코드에서 함수 정의부 위에 '@'로 시작하는 부분이 decorator
- 기본적인 function에 부가적인 장식이라는 의미에서 'decorator'라고 지칭함

In [ ]:
@sample_decorator
def my_function():
    print("hi!")

# sample_decorator가 아직 정의되지 않았기 때문에 위 코드를 실행하면 에러가 납니다

### decorator는 언제 사용할까?
- 예를 들어 아래 `print_hello`라는 함수처럼 인사말을 프린트해주는 간단한 함수가 있을 때 '안녕하세요!'의 바로 뒤에 현재 시간을 함께 프린트해주고 싶다면 어떻게 해야 할까?

In [4]:
def print_hello():
    print("안녕하세요!")

- 아주 간단한 방법으로는 아래처럼 함수 내에 그냥 현재 시간을 찍는 라인을 한 줄 만들어주면 된다

In [7]:
from datetime import datetime
def print_hello():
    print("안녕하세요!")
    print(datetime.now())

- 하지만 이런 함수가 100개쯤 있다면...? 이들 모두에 동일한 라인을 추가하는 것은 귀찮고 효율적이지 못한 방법
- 이 경우 아래와 같이 decorator를 사용할 수 있음
- 아래 예제에서 `print_datetime`이라는 함수를 선언하고 파라미터로 function을 받음(지난 시간에 다루었던 first class 함수)
- 내부에 `decorated`라는 nested 함수를 선언하고 input으로 받은 함수의 앞뒤에 장식적인 라인을 추가해 줌
- 아래와 같은 원리로 동작하므로 입력받은 함수의 중간에 끼어들 수는 없음(입력받은 함수는 통으로 실행되고 그 앞이나 뒤에 라인을 추가해 주는 것)

In [8]:
def print_datetime(func):
    def decorated():
        func()
        print(datetime.now())
    return decorated

@print_datetime
def print_a():
    print("This is A")
    
@print_datetime
def print_b():
    print("This is B")
    
@print_datetime
def print_c():
    print("This is C")
    
print_a()
print_b()
print_c()

This is A
2019-04-27 09:55:10.632344
This is B
2019-04-27 09:55:10.632761
This is C
2019-04-27 09:55:10.633092


### @ 심볼이 없는 경우
- 사실 데코레이터는 아래와 같이 동작하는 것을 `@` 심볼을 사용해 간편하게 만든 것
- first-class function의 특징을 이용해 함수를 parameter로 주고받음
- 아래 예시에서 nested method인 `decorated`에서는 `func`를 입력으로 받지 않았는데도 실행이 가능(closure 함수)

In [44]:
# @ 심볼을 사용하지 않은 데코레이터
def print_datetime(func):
    def decorated():
        return func()
    return decorated

def print_a():
    print("This is A")

decorated_print_a = print_datetime(print_a)
decorated_print_a()

This is A


## 2) function 레벨에서 데코레이터 사용하기: parameter가 있는 경우
- input으로 들어오는 함수의 parameter가 몇 개인지 알 수 없기 때문에 지난 시간에 다루었던 `*args`와 `*kwargs`가 사용됨

In [26]:
def print_datetime(func):
    def decorated(*args, **kwargs):
        func(*args, **kwargs)
        print(datetime.now())
    return decorated

@print_datetime
def print_a(alphabet):
    print("This is A: {}".format(alphabet))
    
@print_datetime
def print_b(alphabet):
    print("This is B: {}".format(alphabet))
    
@print_datetime
def print_c(alphabet):
    print("This is C: {}".format(alphabet))
    
print_a("A")
print_b("B")
print_c("C")

This is A: A
2019-04-27 10:22:35.057534
This is B: B
2019-04-27 10:22:35.057982
This is C: C
2019-04-27 10:22:35.058671


## 3) function 레벨에서 데코레이터 사용하기: 리턴값이 있는 경우

In [34]:
def print_datetime(func):
    def decorated(*args, **kwargs):
        input_alphabet = func(*args, **kwargs)   # return값을 저장한 후
        print(datetime.now())
        return input_alphabet  # 리턴해주는 부분을 추가
    return decorated

@print_datetime
def print_a(alphabet):
    print("This is A: {}".format(alphabet))
    return alphabet
    
@print_datetime
def print_b(alphabet):
    print("This is B: {}".format(alphabet))
    return alphabet
    
@print_datetime
def print_c(alphabet):
    print("This is C: {}".format(alphabet))
    return alphabet
    
val_a = print_a("A")
val_b = print_b("B")
val_c = print_c("C")
print(val_a, val_b, val_c)

This is A: A
2019-04-27 10:33:38.245603
This is B: B
2019-04-27 10:33:38.246049
This is C: C
2019-04-27 10:33:38.246388
A B C


## 4) class 레벨에서 데코레이터 사용하기
- class 형태로 decorator를 사용하고자 한다면 아래 `DatetimeDecorator` 예시처럼 `__call__` 함수로 정의해주면 됨
- 위 function 레벨과 거의 유사하지만 조금 더 명시적임

In [29]:
class DatetimeDecorator:

    def __init__(self, f):
        self.func = f

    def __call__(self, *args, **kwargs):
        self.func(*args, **kwargs)
        print(datetime.now())

        
class MainClass:
    @DatetimeDecorator
    def print_a(alphabet):
        print("This is A: {}".format(alphabet))

    @DatetimeDecorator
    def print_b(alphabet):
        print("This is B: {}".format(alphabet))

    @DatetimeDecorator
    def print_c(alphabet):
        print("This is C: {}".format(alphabet))

        
my = MainClass()
my.print_a("wow")
my.print_b("good")
my.print_c("fantastic!")

This is A: wow
2019-04-27 10:28:36.706975
This is B: good
2019-04-27 10:28:36.707437
This is C: fantastic!
2019-04-27 10:28:36.707798


## 5) 주요 활용 사례
### 분석, 로깅
- 특히 규모가 큰 애플리케이션에서는 현재 무슨 일이 벌어지고 있는지 구체적으로 측정하고 다양한 활동을 정량화하는 지표를 기록해야 할 때가 많음. 데코레이터는 그러한 중요 이벤트를 전용 함수나 메서드에 캡슐화함으로써 이러한 요구사항을 아주 가독성 높고 손쉽게 처리 가능
- 동일한 프린트나 로깅 부분을 일일이 모든 함수에 추가해줄 수 없으므로 로깅 함수를 별도로 지정해 데코레이터로 사용

In [35]:
def log_order_event(func):
    def wrapper(*args, **kwargs):
        print("Ordering: {}".format(func.__name__))
        order = func(*args, **kwargs)
        print("Order result: {}".format(order))
        return order
    return wrapper

@log_order_event
def order_hanburger(*menu_name):
    return menu_name

@log_order_event
def order_pizza(*toppings):
    return toppings
    
pizza_order = order_pizza(['토마토', '치즈'])
burger_order = order_hanburger(['더블패티', '치킨'])

print(pizza_order, burger_order)

Ordering: order_pizza
Order result: (['토마토', '치즈'],)
Ordering: order_hanburger
Order result: (['더블패티', '치킨'],)
(['토마토', '치즈'],) (['더블패티', '치킨'],)


### 유효성 검사와 런타임 검사
- 파이썬의 타입 체계는 타입에 엄격하지만(strongly typed) 매우 동적임. 따라서 들어오고 나가는 데이터에 대해 좀 더 세련된 맞춤형 검사를 강제하여 버그를 방지할 수 있음. 데코레이터를 이용하면 이 모든 작업을 손쉽게 처리하고 한번에 여러 함수에 적용할 수 있음
- 예를 들어 함수가 여러 개 있고 각 함수는 딕셔너리를 하나 반환하는데, 이 딕셔너리에는 다른 필드와 함께 “summary”라는 필드가 포함돼 있음. 이 요약값은 20자를 넘으면 안 되고 이를 위반하면 오류. 아래 예시는 이 같은 오류가 발생할 경우 ValueError를 던지는 데코레이터

In [40]:
def validate_summary(func):
    def wrapper(*args, **kwargs):
        data = func(*args, **kwargs)
        if len(data["summary"]) > 20:
            raise ValueError("Summary too long")
        return data
    return wrapper

@validate_summary
def short_summary1(val):
    return {'summary': "이것은 짧은 summary", 'val': val}

@validate_summary
def short_summary2(val):
    return {'summary': "이것도 짧은 summary", 'val': val}

@validate_summary
def long_summary(val):
    return {'summary': "가나다라마바사아자차카타파하가나다라마바사아자차카타파하", 'val': val}


print(short_summary1("a"))
print(short_summary2("b"))
print(long_summary("c"))   #  오류 발생!

{'val': 'a', 'summary': '이것은 짧은 summary'}
{'val': 'b', 'summary': '이것도 짧은 summary'}


ValueError: Summary too long

### 프레임워크 제작
- 데코레이터를 작성하는 법에 익숙해지고 나면 데코레이터를 사용하는 단순한 문법의 이점을 얻을 수 있음. 바로 사용하기 쉬운 언어에 시맨틱을 추가할 수 있고 파이썬 자체의 문법을 확장할 수 있다는 것
- 사실 많은 오픈소스 프레임워크에서는 데코레이터를 사용. 웹 애플리케이션 프레임워크인 `플라스크(Flask)`에서는 데코레이터를 사용해 HTTP 요청을 처리하는 함수로 URL을 라우팅
- 아래 샘플 코드에서는 `app`이라는 전역 객체와 이 객체의 `route` 함수에서 특정 parameter를 받음. `route`는 핸들러 함수에 적용되는 데코레이터를 반환. 내부적인 동작 원리는 상당히 난해하고 복잡하지만 플라스크를 사용하는 사람의 관점에서는 그 모든 복잡성이 완전히 감춰짐

In [ ]:
# 아래 예시는 flask에서 바로 가져온 것이라 실행은 되지 않습니다!

# For a RESTful todo-list API.
@app.route("/tasks/", methods=["GET"])
def get_all_tasks():
    tasks = app.store.get_all_tasks()
    return make_response(json.dumps(tasks), 200)

@app.route("/tasks/", methods=["POST"])
def create_task():
    payload = request.get_json(force=True)
    task_id = app.store.create_task(
        summary = payload["summary"],
        description = payload["description"],
    )
    task_info = {"id": task_id}
    return make_response(json.dumps(task_info), 201)

@app.route("/tasks/<int:task_id>/")
def task_details(task_id):
    task_info = app.store.task_details(task_id)
    if task_info is None:
        return make_response("", 404)
    return json.dumps(task_info)

- 이 같은 방식으로 데코레이터를 사용하는 것은 일반 파이썬 코드에서도 볼 수 있음. 예를 들어 객체 시스템을 활용할 때 `classmethod`와 `property` 데코레이터가 사용됨
- 이 클래스에는 세 개의 `def` statement가 있고 각각의 시맨틱이 모두 다름
  - 생성자(`__init__`)는 일반 method
  - `for_winter`는 팩터리의 일종을 제공하는 classmethod
  - `progress`는 읽기 전용인 동적 속성
- `@classmethod`와 `@property` 데코레이터의 단순함 덕분에 일상적으로 사용하는 파이썬 객체의 시맨틱을 확장하기가 쉬워짐

In [59]:
class WeatherSimulation:
    def __init__(self, **params):
         self.params = params
            
    @classmethod
    def for_winter(cls, **other_params):
        params = {'month': 'Jan', 'temp': '0'}
        params.update(other_params)
        return cls(**params)

    @property
    def progress(self):
        return self.completed_iterations() / self.total_iterations()

### 재사용이 어려운 코드의 재사용
- 파이썬에서는 표현력 있는 함수 문법, 함수형 프로그래밍 지원, 완전한 기능의 객체 시스템을 통해 코드를 손쉽게 재사용할 수 있는 형태로 캡슐화하는 강력한 도구를 제공합니다. 하지만 이것만으로는 처리할 수 없는 코드 재사용 패턴이 있음
- 불안정한 API를 이용하는 경우를 생각하면, HTTP를 통해 JSON 형식으로 요청을 보내면 99.9%의 경우에는 올바르게 동작함. 하지만 아주 일부 요청이 서버에서 내부 오류를 일으켜서 요청을 재시도해야 하는 경우가 발생. 이 경우 다음과 같은 재시도 로직을 구현할 것

In [ ]:
resp = None
while True:
    resp = make_api_call()
    if resp.status_code == 500 and tries < MAX_TRIES:
        tries += 1
        continue
    break

process_response(resp)

- `make_api_call` 같은 함수가 여러 개 있고 이러한 함수를 코드 곳곳에서 호출한다면, 함수를 호출하는 `while` 반복문을 그때그때마다 구현하기엔 너무 귀찮고 어려움. (새로운 API 호출 함수를 추가할 때마다 같은 작업을 반복...) 이 경우 데코레이터를 사용하면 문제가 상당히 간단해짐

In [ ]:
# 데코레이터가 적용된 함수에서는 Response 객체를 반환하고
# 이 객체에는 status_code 속성이 포함
# 200: 성공, 500: 서버 측 오류
def retry(func):
    def retried_func(*args, **kwargs):
        MAX_TRIES = 3
        tries = 0
        while True:
            resp = func(*args, **kwargs)
            if resp.status_code == 500 and tries < MAX_TRIES:
                tries += 1
                continue
            break
        return resp
    return retried_func


#이렇게 하면 사용하기 쉬운 @retry 데코레이터가 만들어짐
@retry
def make_api_call():
    # ....

## 6) 진짜 decorator의 본질...
### **decorator는 곧 closure. closure란 함수의 함수**

- 아래 예제에서 `hello` 함수는 `message`를 인자로 받은 뒤 다음에 이름을 받을 수 있는 함수인 `greeting` 함수를 반환
- 그런데 `greeting` 함수의 `print(f'{message} - {name}!')`에서 `message` 변수는 `greeting` 안에 없는데 'Hello'가 어떻게 출력될 수 있나?
- 그건 **`greeting` 함수가 선언되는 시점에 `message`라는 변수를 찾을 때 'LEGB 순서' (Local, Enclosing, Global, Built-in) 에 따라 탐색하기 때문(자바스크립트에서는 { } 가 각각의 스코프를 갖지만, 파이썬에서는 LEGB만 네임스페이스를 가질 수 있음)**
- 위 코드에서 `def`가 몇 번 중첩되어 있더라도 계속해서 부모의 부모의 부모의… 함수가 가지는 scope(enclosing)를 탐색할 수 있기 때문에 `message` 변수가 LEGB안에 존재한다면 찾아냄 (코드 내에 아예 `message` 변수가 없으면 에러가 발생)
- 이처럼 **`message`의 값을 기억하고 호출할 때 항상 같은 `message` 값으로 실행되는 환경을 기억하는 함수를 클로저**라고 할 수 있음

In [52]:
def hello(message):
    def greeting(name):
      print('{message} - {name}!'.format(message, name))
      return greeting

    # hello 함수의 리턴값은 def greeting(name): print(...)
    # print(f'Hello - {name}') 함수를 값처럼 가지고 있는 상태
    hello_o_tron = hello('Hello')

    # print('Hello - A!') 함수를 호출
    hello_o_tron('A') # Hello - A!
    hello_o_tron('B') # Hello - B!

### 데코레이터 문법
- 본질적으로는 callable한 객체는 데코레이터로써 사용 가능. 다만 클로저 형태로 짜여진 함수여야 함
- 데코레이터는 클로저의 축약 문법이라고도 함
- 아래 예제는 `PEP-318`에 있는 예제. `synchronized`, `classmethod`는 callable한 객체로  어떠한 일을 수행하는 함수라고 생각하면 됨

In [ ]:
## 아래 예제는 발췌한 샘플이므로 실행되지 않음

def synchronized(lock):
    # codes below...
    pass

def classmethod(arg):
    # codes below...
    pass

def foo(cls):
    # codes below...
    pass

foo = synchronized(lock)(foo)
foo = classmethod(foo)

# 위의 코드와 완전히 동일
# 데코레이터가 중첩된 경우 실행 순서는 아래에서 위 순서임
@classmethod
@synchronized(lock)
def foo(cls):
    pass

- **(참고) closure**: 프로그래밍 언어에서의 클로저란 퍼스트클래스 함수를 지원하는 언어의 네임 바인딩 기술. 클로저는 **어떤 함수를 함수 자신이 가지고 있는 환경과 함께 저장한 레코드. 또한 함수가 가진 프리변수(free variable)를 클로저가 만들어지는 당시의 값과 레퍼런스에 맵핑하여 주는 역할**을 함. 클로저는 일반 함수와는 다르게 자신의 영역 밖에서 호출된 함수의 변수값과 레퍼런스를 복사하고 저장한 뒤, 이 캡처한 값들에 액세스할 수 있게 도와줌
- **(참고) closure의 사용 목적**: 전역변수 사용을 줄이기 위함. 전통적인 프로그래밍에서 한 변수를 특정 함수의 종료 여부와 관계없이 여러 함수에서 사용하려면 보통 변수를 전역변수로 선언하는데, 이러다 보면 간섭현상(side effect)가 발생하기 쉬움. 전역 변수를 사용하면 프로그래밍 자체는 쉬워지나, 사용 빈도수가 높아지면 간섭현상이 생기면서 프로그램에 문제가 생겼을 때 디버깅 하기가 어려워짐. **함수 클로저를 만들기 위해선 내가 함수 클로저로 만들고 싶은 함수를 다른 함수로 감싸면 됨. 이 때 함수 클로저를 감싸주는 함수의 반환형은 함수 클로저**
- **(참고) 프리 변수(free variable)**: 파이썬에서 프리변수는 코드블럭안에서 사용은 되었지만 그 코드블럭안에서 정의되지 않은 변수를 뜻함

In [78]:
# 클로저의 값은 어디에 저장될까...?
def print_datetime(func):
    def decorated():
        return func()
    print("children of decorated: {}".format(dir(decorated)))
    print("\n")
    print("children of decorated.__closure__: {}".format(dir(decorated.__closure__)))
    print("\n")
    print("children of decorated.__closure__[0]: {}".format(dir(decorated.__closure__[0])))
    print("\n")
    print("value of decorated.__closure__[0]: {}".format(decorated.__closure__[0].cell_contents))
    return decorated

def print_a():
    print("This is A")

decorated_print_a = print_datetime(print_a)
decorated_print_a()

children of decorated: ['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


children of decorated.__closure__: ['__add__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'count', 'index']


children of decorated.__closure__[0]: ['__class__', '__delattr__', '__dir__'

## 마무리
- 데코레이터의 장점: 코드 간결성 확보, 중복이 제거되고 재사용성 강화
- 데코레이터의 단점: 지나치게 많이 사용하면 해당 데코레이터가 무슨 역할을 하는지 찾기 어려워 코드 가독성이 떨어짐, 데코레이터가 중첩되어 있으면 디버깅이 어려움
  
- `저는 지금까지 전 세계 곳곳을 여행하면서 수백 명의 소프트웨어 엔지니어가 파이썬을 더 효과적으로 활용하는 법을 가르쳐왔으며, 저의 고급 파이썬 프로그래밍 워크숍에서 배운 내용 가운데 데코레이터를 작성하는 것이 가장 귀중하고 중요한 도구라고 이야기해준 팀이 늘 있었습니다.` (실리콘밸리 파이썬 강사 아론 맥스웰)

## Reference
- [python decorator (데코레이터) 어렵지 않아요](https://bluese05.tistory.com/30)
- [파이썬 데코레이터를 작성하는 법을 배워야 하는 5가지 이유](http://www.hanbit.co.kr/media/channel/view.html?cms_code=CMS5689111564)
- [Python Decorator](https://velog.io/@doondoony/Python-Decorator-101)
- [파이썬 - 클로저 (Closure)](http://schoolofweb.net/blog/posts/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%ED%81%B4%EB%A1%9C%EC%A0%80-closure/)
- [[Python]파이썬의 중첩 함수](https://yes90.tistory.com/50)